In [37]:
from pathlib import Path
import requests
import zipfile
import pandas as pd
import pyarrow.parquet as pq

def download_one_file_of_raw_data(year: int) -> Path:
    """"""
    URL = f'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/bicicletas-publicas/recorridos-realizados-{year}.zip'
    response= requests.get(URL) #, stream=True)

    
    if response.status_code == 200:
        # with open(nombre_archivo_zip, 'wb') as file:
        #     for chunk in response.iter_content(chunk_size=8192):  # Tamaño del búfer ajustado a 8192 bytes
        #         file.write(chunk)
        path = f'../data/raw/recorridos-realizados-{year}.zip'
        open(path, "wb").write(response.content)     
        print(f'descargado año {year}')

        return path
    else:
        raise Exception(f'{URL} is not available')

In [38]:
import pyarrow as pa

def unzip_and_convert_csv_to_parquet(year: int) -> Path:
    nombre_archivo_zip = f"../data/raw/recorridos-realizados-{year}.zip"
        # Descomprimir el archivo zip
    with zipfile.ZipFile(nombre_archivo_zip, 'r') as archivo_zip:

        # Extraer el archivo CSV del zip
        nombre_archivo_csv = archivo_zip.namelist()[0]  # Suponiendo que el archivo CSV es el primer archivo en el zip
        archivo_zip.extractall(f"../data/raw/")

        # Leer el archivo CSV con pandas
        df = pd.read_csv(f"../data/raw/{nombre_archivo_csv}", delimiter=',', decimal=".")

        # Convertir el DataFrame a formato parquet
        nombre_archivo_parquet = f"rides_{year}.parquet"
        table = pa.Table.from_pandas(df)
        pq.write_table(table, f"../data/raw/{nombre_archivo_parquet}")



        # table = pq.Table.from_pandas(df)
        # pq.write_table(table, nombre_archivo_parquet)
        #pq.write_table(pq.Table.from_pandas(df), nombre_archivo_parquet)
        path = f'../data/raw/rides_{year}.parquet'
    return path
    

In [ ]:
# Conversion manual de archivos descargados, ha sido un apaño para resolverlo !!!!!!  importante

# nombre_archivo_zip = f"../data/raw/recorridos-realizados-2025.zip"

#  # Leer el archivo CSV con pandas
# df = pd.read_csv(f"../data/raw/trips_2025.csv", delimiter=',', decimal=".")

# # Convertir el DataFrame a formato parquet
# nombre_archivo_parquet = f"rides_2025.parquet"
# table = pa.Table.from_pandas(df)
# pq.write_table(table, f"../data/raw/{nombre_archivo_parquet}")


# path = f'../data/raw/rides_2025.parquet'

In [3]:
download_one_file_of_raw_data(year=2022)

: 

In [9]:
unzip_and_convert_csv_to_parquet(year=2022)

'../data/raw/rides_2022.parquet'

In [39]:
import pandas as pd

rides = pd.read_parquet('../data/raw/rides_2024.parquet')

rides.head(10)

,id_recorrido,duracion_recorrido,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,id_usuario,modelo_bicicleta,genero
0,20428222,568,2024-01-23 18:36:00,513,308 - SAN MARTIN II,Av. San Martín 5129,-58.490739,-34.597130,2024-01-23 18:45:28,498,055 - HABANA,Gral. José Gervasio Artigas 4298 (y Habana),-58.494959,-34.586598,992557.0,FIT,MALE
1,20431744,1355,2024-01-23 22:41:20,460,133 - BEIRO Y SEGUROLA,Segurola 3194,-58.511930,-34.607500,2024-01-23 23:03:55,382,204 - Biarritz,Biarritz 2403,-58.477255,-34.605431,320782.0,FIT,FEMALE
2,20429936,0,2024-01-23 20:06:22,467,328 - SARMIENTO II,Sarmiento 2037,-58.395893,-34.605514,2024-01-23 20:06:22,6,006 - Parque Lezama,"Avenida Martin Garcia, 295",-58.369758,-34.628526,828678.0,FIT,FEMALE
3,20429976,0,2024-01-23 20:08:17,382,204 - Biarritz,Biarritz 2403,-58.477255,-34.605431,2024-01-23 20:08:17,460,133 - BEIRO Y SEGUROLA,Segurola 3194,-58.511930,-34.607500,320782.0,ICONIC,FEMALE
4,20424802,680,2024-01-23 15:18:39,137,137 - AZOPARDO Y CHILE,AZOPARDO 700,-58.367492,-34.615598,2024-01-23 15:29:59,150,150 - RODRIGO BUENO,Av. España 2200,-58.355465,-34.618755,861425.0,FIT,FEMALE
5,20427241,466,2024-01-23 17:48:05,99,099 - Malabia,780 Malabia & Loyola,-58.435408,-34.596096,2024-01-23 17:55:51,206,159 - Fitz Roy & Gorriti,Fitz Roy & Gorriti,-58.437339,-34.584950,320714.0,FIT,OTHER
6,20431004,1176,2024-01-23 21:17:38,68,105 - PARQUE SAAVEDRA,García del Río 3590,-58.480464,-34.552148,2024-01-23 21:37:14,68,105 - PARQUE SAAVEDRA,García del Río 3590,-58.480464,-34.552148,1041602.0,ICONIC,MALE
7,20431427,1906,2024-01-23 22:03:46,17,017 - Plaza Almagro,"441 Bulnes & Peron, Juan Domingo, Tte. General",-58.418731,-34.606410,2024-01-23 22:35:32,186,186 - MANZANA 66,2875 Moreno,-58.406441,-34.613636,992479.0,FIT,FEMALE
8,20429542,695,2024-01-23 19:44:05,284,229 - RIGLOS,1019 Riglos,-58.435056,-34.631018,2024-01-23 19:55:40,432,187 - Jose Maria Moreno,141 Jose Maria Moreno,-58.435801,-34.619879,320688.0,FIT,FEMALE
9,20431142,492,2024-01-23 21:31:25,432,187 - Jose Maria Moreno,141 Jose Maria Moreno,-58.435801,-34.619879,2024-01-23 21:39:37,284,229 - RIGLOS,1019 Riglos,-58.435056,-34.631018,320688.0,FIT,FEMALE


In [ ]:
import pandas as pd

rides = pd.read_parquet('../data/raw/rides_2022.parquet')

rides.head(10)

,Unnamed: 0,X,Id_recorrido,duracion_recorrido,fecha_origen_recorrido,id_estacion_origen,nombre_estacion_origen,direccion_estacion_origen,long_estacion_origen,lat_estacion_origen,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,direccion_estacion_destino,long_estacion_destino,lat_estacion_destino,id_usuario,modelo_bicicleta,Género
0,1,1,13267975BAEcobici,"2,610",2022-01-16 14:58:42,5BAEcobici,005 - Plaza Italia,Av. Sarmiento 2601,-58.420954,-34.580550,2022-01-16 15:42:12,210BAEcobici,335 - General Urquiza,Figueroa Alcorta & Sarmiento,-58.411278,-34.572165,776361BAEcobici,ICONIC,FEMALE
1,2,2,13268526BAEcobici,545,2022-01-16 17:26:27,51BAEcobici,051 - TUCUMAN,Tucuman & 9 De Julio Av.,-58.382126,-34.601478,2022-01-16 17:35:32,174BAEcobici,174 - MINISTERIO DE EDUCACION,Marcelo T. de Alvear & Rodriguez Peña,-58.391768,-34.597225,776407BAEcobici,ICONIC,MALE
2,3,3,13268400BAEcobici,"2,061",2022-01-16 16:51:12,161BAEcobici,161 - Humahuaca,3912 Humahuaca,-58.419676,-34.602078,2022-01-16 17:25:33,117BAEcobici,117 - HUMBERTO 1°,Peru 1016,-58.374176,-34.620101,671762BAEcobici,ICONIC,FEMALE
3,4,4,13268164BAEcobici,"12,748",2022-01-16 15:58:01,210BAEcobici,335 - General Urquiza,Figueroa Alcorta & Sarmiento,-58.411278,-34.572165,2022-01-16 19:30:29,382BAEcobici,204 - Biarritz,Biarritz 2403,-58.477390,-34.605470,776361BAEcobici,ICONIC,FEMALE
4,5,5,13270010BAEcobici,"4,337",2022-01-16 23:40:09,215BAEcobici,113 - Guatemala,Guatemala 4773,-58.424996,-34.585878,2022-01-17 00:52:26,205BAEcobici,125 - F.J.Santamaria de Oro,F.J.Santamaria de Oro & Guatemala,-58.428016,-34.583323,454615BAEcobici,ICONIC,OTHER
5,6,6,13269548BAEcobici,"4,243",2022-01-16 20:42:30,268BAEcobici,399 - GARCIA DEL RIO,Av. García del Río 3182,-58.477000,-34.550300,2022-01-16 21:53:13,268BAEcobici,399 - GARCIA DEL RIO,Av. García del Río 3182,-58.477000,-34.550300,200959BAEcobici,ICONIC,OTHER
6,7,7,13268959BAEcobici,932,2022-01-16 18:47:17,278BAEcobici,233 - MONROE,2519 Superi,-58.469813,-34.564122,2022-01-16 19:02:49,236BAEcobici,254 - Plaza Rafael Hernandez,Vuelta de Obligado 2004,-58.455166,-34.562161,823366BAEcobici,ICONIC,FEMALE
7,8,8,13267669BAEcobici,"4,337",2022-01-16 12:55:26,368BAEcobici,378 - AGRONOMIA,4351 San Martin,-58.482079,-34.598070,2022-01-16 14:07:43,368BAEcobici,378 - AGRONOMIA,4351 San Martin,-58.482079,-34.598070,826754BAEcobici,ICONIC,MALE
8,9,9,13267592BAEcobici,"3,735",2022-01-16 12:25:03,382BAEcobici,204 - Biarritz,Biarritz 2403,-58.477390,-34.605470,2022-01-16 13:27:18,5BAEcobici,005 - Plaza Italia,Av. Sarmiento 2601,-58.420954,-34.580550,776361BAEcobici,ICONIC,FEMALE
9,10,10,13376812BAEcobici,366,2022-01-29 15:13:01,433BAEcobici,273 - Plazoleta Colombia,1619 Brandsen,-58.373726,-34.637697,2022-01-29 15:19:07,6BAEcobici,006 - Parque Lezama,"Avenida Martin Garcia, 295",-58.369758,-34.628526,772370BAEcobici,ICONIC,MALE


In [40]:
rides.shape

(3559284, 17)

In [41]:
rides.id_estacion_origen.nunique()

395

In [ ]:
#rides[['fecha_origen_recorrido', 'id_estacion_origen']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3559284 entries, 0 to 3559283
Data columns (total 2 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   fecha_origen_recorrido  object
 1   id_estacion_origen      int64 
dtypes: int64(1), object(1)
memory usage: 54.3+ MB


In [31]:
#Nos quedamos sólo con las columnas que nos interesan y las renombramos
rides = rides[['fecha_origen_recorrido', 'id_estacion_origen']]
# Eliminar la parte "BAEcobici" y convertir a tipo int
rides['id_estacion_origen'] = rides['id_estacion_origen'].str.replace('BAEcobici', '').astype(int)

rides.rename(columns={
    'fecha_origen_recorrido': 'pickup_datetime',
    'id_estacion_origen': 'pickup_location_id',
}, inplace=True)

rides['pickup_datetime'] = pd.to_datetime(rides['pickup_datetime'],format='%Y-%m-%d %H:%M:%S')
rides.head(10)

,pickup_datetime,pickup_location_id
0,2024-01-23 18:36:00,513
1,2024-01-23 22:41:20,460
2,2024-01-23 20:06:22,467
3,2024-01-23 20:08:17,382
4,2024-01-23 15:18:39,137
5,2024-01-23 17:48:05,99
6,2024-01-23 21:17:38,68
7,2024-01-23 22:03:46,17
8,2024-01-23 19:44:05,284
9,2024-01-23 21:31:25,432


In [32]:
rides.pickup_location_id.nunique()

395

In [33]:
rides['pickup_datetime'].describe()

C:\Users\jayan\AppData\Local\Temp\ipykernel_22740\3389344848.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides['pickup_datetime'].describe()


count                 3559284
unique                3227772
top       2024-01-24 19:37:35
freq                       13
first     2024-01-01 00:06:50
last      2024-12-31 23:58:25
Name: pickup_datetime, dtype: object

In [34]:
rides = rides[rides.pickup_datetime >= '2024-01-01']
rides = rides[rides.pickup_datetime < '2025-01-01']
rides['pickup_datetime'].describe()

C:\Users\jayan\AppData\Local\Temp\ipykernel_22740\4188331117.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides['pickup_datetime'].describe()


count                 3559284
unique                3227772
top       2024-01-24 19:37:35
freq                       13
first     2024-01-01 00:06:50
last      2024-12-31 23:58:25
Name: pickup_datetime, dtype: object

In [35]:
rides.to_parquet('../data/transformed/validated_rides_2024.parquet')